In [1]:
import _pickle as pickle
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import copy
import glob
import gc
import sys

In [2]:
%load_ext autoreload
%autoreload 2

In [43]:
pd.set_option('max_rows',500,'max_colwidth',500)

In [3]:
import ipdb

In [ ]:
aui_info = []

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRCONSO.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        aui = line[7]
        string = line[-5]
        
        aui_info.append({'AUI':aui, 'CUI':cui, 'STR':string})
        
cui2sg = {}

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRSTY.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        sg = line[3]
        cui2sg[cui] = sg
        
cui2aui = {}
aui2cui = {}
aui2str = {}
aui2sg = {}

cui_sg = []
cui_aui = []

for tup in tqdm(aui_info):
    aui = tup['AUI']
    
    cui = tup['CUI']
    string = tup['STR']
    sg = cui2sg[cui]

    auis = cui2aui.get(cui, [])
    auis.append(aui)
    cui2aui[cui] = auis

    aui2cui[aui] = cui
    aui2str[aui] = string
    aui2sg[aui] = sg

    cui_sg.append((cui, sg))
    cui_aui.append((cui, aui))
        
semgroups = pd.read_csv('SemGroups.txt',sep='|',header=None)

semtype2sg = {}

for i, row in semgroups.iterrows():
    
    st = row[3]
    sg = row[1]
    
    semtype2sg[st] = sg

In [ ]:
with open('/data/Bodenreider_UMLS_DL/Interns/Bernal/mrconso.csv','w') as f:
    for aui_dict in aui_info:
        f.write(aui_dict['AUI']+'\t'+aui_dict['CUI']+'\t'+aui_dict['STR']+'\n')

In [ ]:
original_umls = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/INTERSECT_AUI2ID.PICKLE','rb'))
new_auis = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/UNIQUE_AUI2ID.PICKLE','rb'))

aui_vecs  = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/AUI2LAYER.PICKLE','rb'))

In [ ]:
from word_trie import Trie

In [ ]:
trie = Trie()

for aui in tqdm(original_umls):
    trie.insert(aui2str[aui])
    
for aui in tqdm(new_auis):
    trie.insert(aui2str[aui])

In [ ]:
len(trie.root)

In [ ]:
sys.getsizeof(trie)

In [ ]:
# pickle.dump(trie, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls_2020AB_trie.p','wb'))

In [ ]:
abstract_files = glob.glob('/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/raw_abstracts/pubmed22n*')

In [ ]:
pickle.dump(abstract_files,open('pubmed_file_list.p','wb'))

In [ ]:
num_abstracts = 0
abstracts = []

for file in tqdm(abstract_files):
    
    direct = '/'.join(file.split('/')[:-2]) + '/umls_mentions_2020AB'
    file_id = file.split('/')[-1].split('.')[0]
    
    if not(os.path.exists(direct)):
        os.makedirs(direct)
    
    new_file = direct + '/' + file_id + '.p'
            
    abstract_info = pickle.load(open(file,'rb'))

    num_abstracts += len(abstract_info[0])

    for title, date, abstract in zip(abstract_info[0],abstract_info[1],abstract_info[2]):

        abstract = abstract[0]
        
        if abstract is not None:
            abstracts.append(abstract)

In [ ]:
len(abstracts)

In [ ]:
num_abstracts = 0

for file in tqdm(abstract_files):
    
    direct = '/'.join(file.split('/')[:-2]) + '/umls_mentions_2020AB'
    file_id = file.split('/')[-1].split('.')[0]
    
    if not(os.path.exists(direct)):
        os.makedirs(direct)
    
    new_file = direct + '/' + file_id + '.p'
    
    if not(os.path.exists(new_file)):
        
        abstract_info = pickle.load(open(file,'rb'))

        num_abstracts += len(abstract_info[0])

        auis_per_abstract = []

        for title, date, abstract in zip(abstract_info[0],abstract_info[1],abstract_info[2]):

            abstract = abstract[0]
            abstract_auis = []

            if abstract is not None:

                tokenized_abstract = abstract.split()

                for i, word in enumerate(tokenized_abstract):

                    word_in_dict = trie.startsWith(word)

                    if word_in_dict:

                        j=1

                        phrase = ' '.join(tokenized_abstract[i:i+j])
                        phrase_in_dict = trie.startsWith(phrase)

                        while phrase_in_dict and i+j < len(tokenized_abstract):
                            if 'END' in phrase_in_dict:
                                abstract_auis.append((phrase, i, i+j))

                            j += 1

                            phrase = ' '.join(tokenized_abstract[i:i+j])
                            phrase_in_dict = trie.startsWith(phrase)                

                auis_per_abstract.append(abstract_auis)
            else:
                auis_per_abstract.append(None)        

        pickle.dump(auis_per_abstract, open(new_file,'wb'))

In [ ]:
num_abstracts

In [ ]:
titles, dates_per_article, abstracts_per_article = abstract_info

In [5]:
abstract_mention_files = glob.glob('/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/*.p')

In [6]:
abstract_mention_files

['/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0321.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0014.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0981.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0642.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0085.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0270.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0076.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0460.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0018.p',
 '/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_abstracts/umls_mentions_2020AB/pubmed22n0177.p',
 '/data/Bo

In [ ]:
docs = []

for file in tqdm(abstract_mention_files):
    mentions_per_abstract = pickle.load(open(file,'rb'))

    for mentions in mentions_per_abstract:
        if mentions != None:
            docs.append(mentions)

In [6]:
index = {}

for file in tqdm(abstract_mention_files):
    
    abs_index = {}
    
    mentions_per_abstract = pickle.load(open(file,'rb'))
    
    for abs_num, mentions in enumerate(mentions_per_abstract):
        if mentions != None:
            
            for mention,_,_ in mentions:
                
                term_abs_nums = abs_index.get(mention, [])
                term_abs_nums.append(abs_num)
                abs_index[mention] = term_abs_nums
                
    index[file] = abs_index

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1114/1114 [27:50<00:00,  1.50s/it]


In [7]:
index_per_term = {}

for file in tqdm(index.keys()):
    
    file_index = index[file]
    
    for term in file_index.keys():
        term_file_index = file_index[term]
        
        index_per_term_files = index_per_term.get(term, {})
        index_per_term_files[file] = term_file_index
        index_per_term[term] = index_per_term_files

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1114/1114 [01:31<00:00, 12.14it/s]


In [8]:
len(index_per_term.keys())

765425

In [9]:
# pickle.dump(index_per_term, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/index_per_term_and_file_pubmed2022.p','wb'))

In [7]:
index_per_term = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/index_per_term_and_file_pubmed2022.p','rb'))

In [32]:
sapbert_errors_full = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls_2020AB_SAPBERT_errors.csv')

In [5]:
sapbert_errors = sapbert_errors[(sapbert_errors['sapbert_errors'] != '[]') & pd.notna(sapbert_errors['sapbert_errors'])]

In [10]:
file_size = []

for file in tqdm(np.random.permutation(abstract_mention_files)[:100]):
        
    mentions_per_abstract = pickle.load(open(file,'rb'))
    
    file_size.append(sys.getsizeof(mentions_per_abstract))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:35<00:00,  1.05it/s]


In [11]:
mean_size = np.mean(file_size)
std = np.std(file_size)

threshold = mean_size

In [13]:
large_docs = {}
new_candidate_col = []

for term, errors in tqdm(zip(sapbert_errors.strings, sapbert_errors.sapbert_errors),total = len(sapbert_errors)):
        
    cooccur_candidates = []
    
    if term in index_per_term:
        index_by_file = index_per_term[term]

        for file in index_by_file.keys():

            if file not in large_docs:
                mentions_per_abstract = pickle.load(open(file,'rb'))
                
                if sys.getsizeof(mentions_per_abstract) > threshold:
                    large_docs[file] = mentions_per_abstract
            else:
                mentions_per_abstract = large_docs[file]
            
            abstracts_for_term = index_by_file[file]

            for abstract_num in abstracts_for_term:
                cooccur_candidates.extend(mentions_per_abstract[abstract_num])
                
            if len(cooccur_candidates) > 1000:
                break

        new_candidate_col.append(cooccur_candidates)
    else:
        new_candidate_col.append(cooccur_candidates)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40589/40589 [1:30:56<00:00,  7.44it/s]


In [15]:
# pickle.dump(new_candidate_col, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_candidates.p','wb'))

In [6]:
new_candidate_col = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/pubmed_candidates.p','rb'))

In [8]:
sapbert_errors['pubmed_candidates'] = new_candidate_col

In [11]:
sapbert_errors['num_pubmed_candidates'] = [len(c) for c in sapbert_errors.pubmed_candidates]

In [13]:
sapbert_errors[sapbert_errors.num_pubmed_candidates > 0]

,Unnamed: 0,num_pubmed_candidates
count,1.117500e+04,1.117500e+04
mean,8.592158e+06,1.828556e+03
std,4.598224e+04,5.424073e+04
min,8.521225e+06,3.000000e+00
25%,8.558285e+06,2.060000e+02
50%,8.593404e+06,9.410000e+02
75%,8.610900e+06,1.125000e+03
max,8.738396e+06,5.637837e+06


In [14]:
len(sapbert_errors)

40589

In [17]:
fixes = []
new_errors = []

for i,row in sapbert_errors.iterrows():
    
    errors = row['sapbert_errors']
    errors = errors.split('|||')
    
    pm_candidates = row['pubmed_candidates']
    pm_candidates = set([c[0] for c in pm_candidates])
    
    fixed_errors = []
    remaining_errors = []
    
    for error in errors:
        if error in pm_candidates:
            fixed_errors.append(error)
        else:
            remaining_errors.append(error)
            
    fixes.append(fixed_errors)
    new_errors.append(remaining_errors)

In [18]:
sapbert_errors['fixed_errors'] = fixes
sapbert_errors['remaining_errors'] = new_errors

In [21]:
sapbert_errors['num_remaining_errors'] = [len(e) for e in sapbert_errors['remaining_errors']]
sapbert_errors['num_errors'] = [len(e.split('|||')) for e in sapbert_errors['sapbert_errors']]

In [44]:
sapbert_errors[sapbert_errors.num_errors > sapbert_errors.num_remaining_errors][:200]

,Unnamed: 0,strings_x,auis_x,2020AA_synonyms_x,synonym_strings_x,sapbert_errors_x,strings_y,auis_y,2020AA_synonyms_y,synonym_strings_y,sapbert_errors_y,pubmed_candidates,num_pubmed_candidates,fixed_errors,remaining_errors,num_remaining_errors,num_errors
17849,8588020,Finkel-Biskis-Reilly murine sarcoma virus (FBR-MuSV) ubiquitously expressed,A31827892,"['A20790432', 'A6912330', 'A12035262', 'A20808512', 'A30286510', 'A20754548', 'A20682983', 'A11963778', 'A24598819', 'A20790434', 'A20790435', 'A20790433', 'A20754549', 'A20736604', 'A20736602', 'A20736603']","['FAU gene', 'FAU gene', 'FAU GENE', 'ribosomal protein S30', 'FAU ubiquitin like and ribosomal protein S30 fusion', 'RPS30', 'FAU', 'FAU', 'FBR-MuSV ASSOCIATED UBIQUITOUSLY EXPRESSED GENE', 'Fubi', 'MNSFbeta', 'FLJ22986', 'Fub1', 'Monoclonal nonspecific suppressor factor beta', 'asr1', 'S30']",FBR-MuSV ASSOCIATED UBIQUITOUSLY EXPRESSED GENE|||MNSFbeta|||RPS30|||asr1|||Fubi|||FAU gene|||S30|||Monoclonal nonspecific suppressor factor beta|||FAU ubiquitin like and ribosomal protein S30 fusion|||FAU GENE|||FAU|||Fub1|||ribosomal protein S30|||FLJ22986,Finkel-Biskis-Reilly murine sarcoma virus (FBR-MuSV) ubiquitously expressed,A31827892,"['A20790432', 'A6912330', 'A12035262', 'A20808512', 'A30286510', 'A20754548', 'A20682983', 'A11963778', 'A24598819', 'A20790434', 'A20790435', 'A20790433', 'A20754549', 'A20736604', 'A20736602', 'A20736603']","['FAU gene', 'FAU gene', 'FAU GENE', 'ribosomal protein S30', 'FAU ubiquitin like and ribosomal protein S30 fusion', 'RPS30', 'FAU', 'FAU', 'FBR-MuSV ASSOCIATED UBIQUITOUSLY EXPRESSED GENE', 'Fubi', 'MNSFbeta', 'FLJ22986', 'Fub1', 'Monoclonal nonspecific suppressor factor beta', 'asr1', 'S30']",FBR-MuSV ASSOCIATED UBIQUITOUSLY EXPRESSED GENE|||MNSFbeta|||RPS30|||asr1|||Fubi|||FAU gene|||S30|||Monoclonal nonspecific suppressor factor beta|||FAU ubiquitin like and ribosomal protein S30 fusion|||FAU GENE|||FAU|||Fub1|||ribosomal protein S30|||FLJ22986,"[(Finkel-Biskis-Reilly murine sarcoma virus, 0, 4), (Finkel-Biskis-Reilly murine sarcoma virus (FBR-MuSV) ubiquitously expressed, 0, 7), (murine, 1, 2), (murine sarcoma virus, 1, 4), (sarcoma, 2, 3), (sarcoma virus, 2, 4), (virus, 3, 4), (gene, 8, 9), (is, 9, 10), (in, 11, 12), (human, 12, 13), (breast, 14, 15), (and, 15, 16), (ovarian, 16, 17), (is, 21, 22), (associated, 22, 23), (associated with, 22, 24), (poor, 24, 25), (poor prognosis, 24, 26), (prognosis, 25, 26), (in, 26, 27), (breast,...",106.0,"[FAU gene, FAU]","[FBR-MuSV ASSOCIATED UBIQUITOUSLY EXPRESSED GENE, MNSFbeta, RPS30, asr1, Fubi, S30, Monoclonal nonspecific suppressor factor beta, FAU ubiquitin like and ribosomal protein S30 fusion, FAU GENE, Fub1, ribosomal protein S30, FLJ22986]",12.0,14.0
31172,8632238,Camptodactyly-arthropathy-coxa vara-pericarditis syndrome,A31789764,"['A12032467', 'A24585854', 'A12057999', 'A18464989', 'A30224281', 'A11989583', 'A30264096', 'A30194645', 'A30234380', 'A24591096', 'A18467983', 'A30224282', 'A20976385', 'A24566253', 'A30254264', 'A18458836', 'A24595266', 'A18471132', 'A24620171', 'A24595267', 'A30244146', 'A18464988', 'A20980616', 'A24599427', 'A20976384', 'A24616041', 'A18452610', 'A18471131', 'A20980615', 'A24595265']","['CAMPTODACTYLY-ARTHROPATHY-COXA VARA-PERICARDITIS SYNDROME', 'CAMPTODACTYLY-ARTHROPATHY-COXA VARA-PERICARDITIS SYNDROME', 'CAMPTODACTYLY-ARTHROPATHY-COXA VARA-PERICARDITIS SYNDROME', 'Camptodactyly-arthropathy-coxa vara-pericarditis syndrome', 'Camptodactyly, arthropathy, coxa-vara, pericarditis syndrome', 'CACP', 'CACP syndrome', 'Camptodactyly, arthropathy, coxa-vara, pericarditis syndrome (disorder)', 'CACP (camptodactyly, arthropathy, coxa-vara, pericarditis) syndrome', 'CAP SYNDROME', ...","CACP|||Jacobs syndrome|||FIBROSING SEROSITIS, FAMILIAL|||JACOBS SYNDROME|||Fibrosing serositis, familial",Camptodactyly-arthropathy-coxa vara-pericarditis syndrome,A31789764,"['A12032467', 'A24585854', 'A12057999', 'A18464989', 'A30224281', 'A11989583', 'A30264096', 'A30194645',

In [30]:
len(sapbert_errors)

40589

In [26]:
sapbert_errors.num_errors.sum()

159261

In [27]:
sapbert_errors.num_remaining_errors.sum()

157205

In [40]:
sapbert_errors = sapbert_errors_full.merge(sapbert_errors, on='Unnamed: 0', how='outer')

In [41]:
sapbert_errors.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_errors_with_pubmed_candidates.csv')

In [42]:
sapbert_errors

,Unnamed: 0,strings_x,auis_x,2020AA_synonyms_x,synonym_strings_x,sapbert_errors_x,strings_y,auis_y,2020AA_synonyms_y,synonym_strings_y,sapbert_errors_y,pubmed_candidates,num_pubmed_candidates,fixed_errors,remaining_errors,num_remaining_errors,num_errors
0,8768549,TRIFOLIUM PRATENSE FLOWER 3 [hp_X] in 1 mL / C...,A31798620,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8769337,LILIUM LANCIFOLIUM WHOLE FLOWERING 15 [hp_X] i...,A31791529,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8949912,GARLIC 3 [hp_X] in 1 mL / AVENA SATIVA FLOWERI...,A32332182,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8763774,SUS SCROFA PITUITARY GLAND 6 [hp_X] in 1 mL / ...,A31622453,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8764953,CHELIDONIUM MAJUS WHOLE 3 [hp_X] in 1 mL / LOB...,A31758369,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430130,8529453,vincristine,A31682108,"['A0491552', 'A18576062', 'A20895977', 'A22722...","['vincristine', 'vincristine', 'vincristine', ...",lcr|||LCR,vincristine,A31682108,"['A0491552', 'A18576062', 'A20895977', 'A22722...","['vincristine', 'vincristine', 'vincristine', ...",lcr|||LCR,"[(In, 0, 1), (P-GP, 8, 9), (or, 9, 10), (MDR, ...",2108.0,[],"[lcr, LCR]",2.0,2.0
430131,8529270,uridine,A31751560,"['A0491264', 'A18613289', 'A19986797', 'A26682...","['uridine', 'uridine', 'uridine', 'uridine', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430132,8529351,Vagina,A31590388,"['A0131175', 'A0131181', 'A15368227', 'A184069...","['Vagina', 'Vagina', 'Vagina', 'Vagina', 'Vagi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430133,8601787,SAV,A11924706,"['A20745789', 'A6908930', 'A23821695', 'A23809...","['SAV1 gene', 'SAV1 gene', 'WW domain-containi...",WWP4|||WW domain-containing adaptor 45|||WW45|...,SAV,A11924706,"['A20745789', 'A6908930', 'A23821695', 'A23809...","['SAV1 gene', 'SAV1 gene', 'WW domain-containi...",WWP4|||WW domain-containing adaptor 45|||WW45|...,"[(Sequence, 0, 1), (data, 1, 2), (E2, 8, 9), (...",1036.0,[],"[WWP4, WW domain-containing adaptor 45, WW45, ...",4.0,4.0


In [45]:
aui_info = []

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AA-ACTIVE/META/MRCONSO.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        aui = line[7]
        string = line[-5]
        
        aui_info.append({'AUI':aui, 'CUI':cui, 'STR':string})
        
cui2sg = {}

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AA-ACTIVE/META/MRSTY.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        sg = line[3]
        cui2sg[cui] = sg
        
cui2aui = {}
aui2cui = {}
aui2str = {}
aui2sg = {}

cui_sg = []
cui_aui = []

for tup in tqdm(aui_info):
    aui = tup['AUI']
    
    cui = tup['CUI']
    string = tup['STR']
    sg = cui2sg[cui]

    auis = cui2aui.get(cui, [])
    auis.append(aui)
    cui2aui[cui] = auis

    aui2cui[aui] = cui
    aui2str[aui] = string
    aui2sg[aui] = sg

    cui_sg.append((cui, sg))
    cui_aui.append((cui, aui))
        
semgroups = pd.read_csv('SemGroups.txt',sep='|',header=None)

semtype2sg = {}

for i, row in semgroups.iterrows():
    
    st = row[3]
    sg = row[1]
    
    semtype2sg[st] = sg

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14770662/14770662 [02:01<00:00, 121647.96it/s]


In [ ]:
aui2str['A31820876']